### CSV and Excel files - Structured Data

In [26]:
import pandas as pd
import os

In [27]:
os.makedirs('data/structured_files', exist_ok=True)

In [28]:
data = {
    'Product': ['Laptop', 'Smartphone', 'Tablet'],
    'Price': [1200, 800, 400],
    'Stock': [50, 150, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD.',
        'Latest model smartphone with excellent camera features.',
        'Compact tablet with long battery life and vibrant display.'
    ]
}

#save as CSV

df = pd.DataFrame(data)
csv_path = 'data/structured_files/products.csv'
df.to_csv(csv_path, index=False)
print(f"CSV file saved at: {csv_path}")

CSV file saved at: data/structured_files/products.csv


In [29]:
# Save as Excel with multiple sheets
with pd.ExcelWriter('data/structured_files/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)
    
    #Add another sheet
    summary_data = {
        'Category': ['Electronics'],
        'Total Products': [len(data['Product'])],
        'Total Stock': [sum(data['Stock'])]
    }

    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

## CSV Processing

In [30]:
from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader

In [31]:
# CSV Loader
csv_loader = CSVLoader('data/structured_files/products.csv',
                       encoding='utf-8',
                       csv_args={'delimiter': ',', 'quotechar': '"'})

csv_documents = csv_loader.load()
print(f"Number of documents loaded from CSV: {len(csv_documents)}")
print(f"First document content:\n{csv_documents[0].page_content}")
print(f'Metadata:\n{csv_documents[0].metadata}')
print('---')
print(csv_documents)

Number of documents loaded from CSV: 3
First document content:
Product: Laptop
Price: 1200
Stock: 50
Description: High-performance laptop with 16GB RAM and 512GB SSD.
Metadata:
{'source': 'data/structured_files/products.csv', 'row': 0}
---
[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nPrice: 1200\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD.'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Smartphone\nPrice: 800\nStock: 150\nDescription: Latest model smartphone with excellent camera features.'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Tablet\nPrice: 400\nStock: 100\nDescription: Compact tablet with long battery life and vibrant display.')]


In [36]:
import pandas as pd
from typing import List
from langchain_core.documents import Document

def process_csv_intelligently(filepath: str) -> List[Document]:
    """Process CSV file intelligently based on its structure."""
    
    df = pd.read_csv(filepath)
    
    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()
    
    documents = []

    for idx, row in df.iterrows():
        # Build structured content safely
        content = f"""Product Information:
        Name: {row.get('product', 'N/A')}
        Description: {row.get('description', 'N/A')}
        Price: {row.get('price', 'N/A')}
        Category: {row.get('category', 'N/A')}
        """
        
        # Create document with metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row.get('product', 'N/A'),
                'category': row.get('category', 'N/A'),
                'price': row.get('price', 'N/A'),
                'data_type': 'product_info'
            }
        )
        documents.append(doc)

    return documents


In [37]:
process_csv_intelligently('data/structured_files/products.csv')

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'N/A', 'price': 1200, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Description: High-performance laptop with 16GB RAM and 512GB SSD.\n        Price: 1200\n        Category: N/A\n        '),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 1, 'product_name': 'Smartphone', 'category': 'N/A', 'price': 800, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Smartphone\n        Description: Latest model smartphone with excellent camera features.\n        Price: 800\n        Category: N/A\n        '),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 2, 'product_name': 'Tablet', 'category': 'N/A', 'price': 400, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Tablet\n        Description: Compact table

### Excel Processing

In [38]:
# Method 1: Using pandas for full control
print("1️⃣ Pandas-based Excel Processing")
def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel with sheet awareness"""
    documents = []
    
    # Read all sheets
    excel_file = pd.ExcelFile(filepath)
    
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)
        
        # Create document for each sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)
        
        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': filepath,
                'sheet_name': sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns),
                'data_type': 'excel_sheet'
            }
        )
        documents.append(doc)
    
    return documents

1️⃣ Pandas-based Excel Processing


In [39]:
excel_docs = process_excel_with_pandas('data/structured_files/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")

Processed 2 sheets


In [40]:
excel_docs

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 3, 'num_columns': 4, 'data_type': 'excel_sheet'}, page_content='Sheet: Products\nColumns: Product, Price, Stock, Description\nRows: 3\n\n   Product  Price  Stock                                                Description\n    Laptop   1200     50       High-performance laptop with 16GB RAM and 512GB SSD.\nSmartphone    800    150    Latest model smartphone with excellent camera features.\n    Tablet    400    100 Compact tablet with long battery life and vibrant display.'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 1, 'num_columns': 3, 'data_type': 'excel_sheet'}, page_content='Sheet: Summary\nColumns: Category, Total Products, Total Stock\nRows: 1\n\n   Category  Total Products  Total Stock\nElectronics               3          300')]

In [51]:
from langchain_community.document_loaders import UnstructuredExcelLoader
# Method 2: UnstructuredExcelLoader
print("\n2️⃣ UnstructuredExcelLoader")
try:
    excel_loader = UnstructuredExcelLoader(
        'data/structured_files/inventory.xlsx',
        mode="elements"
    )
    unstructured_docs = excel_loader.load()
    print(unstructured_docs)
except Exception as e:
    print("  ℹ️ Errror")


2️⃣ UnstructuredExcelLoader
  ℹ️ Errror


NameError: name 'unstructured_docs' is not defined